In [2]:
import os
import yaml
import pandas as pd
import sqlalchemy

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display 
pd.options.display.max_columns = None
%matplotlib inline

def connect(poolclass=sqlalchemy.pool.QueuePool):
    with open(os.path.join(os.path.join('../..','config'), 'donors_db_profile.yaml')) as fd:
        config = yaml.load(fd)
        dburl = sqlalchemy.engine.url.URL(
            "postgres",
            host=config["host"],
            username=config["user"],
            database=config["db"],
            password=config["pass"],
            port=config["port"],
        )
        return sqlalchemy.create_engine(dburl, poolclass=poolclass)

    
conn = connect()

import RecallAdjuster as ra
from importlib import reload
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns

/home/hemank/.pyenv/versions/3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  from ipykernel import kernelapp as app


In [3]:
base = datetime.datetime.strptime('2013-03-01', '%Y-%m-%d')   #Corresponding to latest train_end_time\n",
date_pairs = []
for x in range(16, -1, -1):
    date_pairs.append(
        (
        (base - relativedelta(months=2*x) - relativedelta(months=2)).strftime('%Y-%m-%d'),
        (base - relativedelta(months=2*x) - relativedelta(months=2)).strftime('%Y-%m-%d')
        )
    )
    date_pairs.append(
        (
        (base - relativedelta(months=2*x) - relativedelta(months=2)).strftime('%Y-%m-%d'),
        (base - relativedelta(months=2*x)).strftime('%Y-%m-%d')
        )
    )
print(date_pairs)

[('2010-05-01', '2010-05-01'), ('2010-05-01', '2010-07-01'), ('2010-07-01', '2010-07-01'), ('2010-07-01', '2010-09-01'), ('2010-09-01', '2010-09-01'), ('2010-09-01', '2010-11-01'), ('2010-11-01', '2010-11-01'), ('2010-11-01', '2011-01-01'), ('2011-01-01', '2011-01-01'), ('2011-01-01', '2011-03-01'), ('2011-03-01', '2011-03-01'), ('2011-03-01', '2011-05-01'), ('2011-05-01', '2011-05-01'), ('2011-05-01', '2011-07-01'), ('2011-07-01', '2011-07-01'), ('2011-07-01', '2011-09-01'), ('2011-09-01', '2011-09-01'), ('2011-09-01', '2011-11-01'), ('2011-11-01', '2011-11-01'), ('2011-11-01', '2012-01-01'), ('2012-01-01', '2012-01-01'), ('2012-01-01', '2012-03-01'), ('2012-03-01', '2012-03-01'), ('2012-03-01', '2012-05-01'), ('2012-05-01', '2012-05-01'), ('2012-05-01', '2012-07-01'), ('2012-07-01', '2012-07-01'), ('2012-07-01', '2012-09-01'), ('2012-09-01', '2012-09-01'), ('2012-09-01', '2012-11-01'), ('2012-11-01', '2012-11-01'), ('2012-11-01', '2013-01-01'), ('2013-01-01', '2013-01-01'), ('2013-01

In [ ]:
query = Template("""
WITH mg_rns AS (
  SELECT *,
         row_number() OVER (PARTITION BY train_end_time, list_size, metric, parameter ORDER BY base_value DESC, base_max_recall_ratio ASC, RANDOM()) AS rn_base,
         row_number() OVER (PARTITION BY train_end_time, list_size, metric, parameter ORDER BY adj_value DESC, adj_max_recall_ratio ASC, RANDOM()) AS rn_adj
  FROM {{schema}}.model_adjustment_results_plevel
  WHERE past_train_end_time = train_end_time
)
, base_mgs AS (
  SELECT * FROM mg_rns WHERE rn_base = 1
)
, adj_mgs AS (
  SELECT * FROM mg_rns WHERE rn_adj = 1
)

-- Simple model selection on last time period, baseline with no recall adjustments
SELECT 'Best Unadjusted Metric - Unadjusted'::VARCHAR(128) AS strategy,
       r.train_end_time, r.past_train_end_time,
       r.list_size, r.metric, r.parameter,
       r.base_value AS value,
       r.base_max_recall_ratio AS max_recall_ratio,
       r.base_recall_white_to_nonwhite AS recall_w_to_nw,
       r.base_recall_nonwhite_to_white AS recall_nw_to_w
FROM {{schema}}.model_adjustment_results_plevel r
JOIN base_mgs b
  ON r.model_group_id = b.model_group_id
  AND r.past_train_end_time = b.train_end_time
  AND r.list_size = b.list_size
  AND r.metric = b.metric
  AND r.parameter = b.parameter
WHERE r.train_end_time > r.past_train_end_time

UNION ALL

-- Model selection on last time before adjustment, with adjustment applied
SELECT 'Best Unadjusted Metric - Adjusted'::VARCHAR(128) AS strategy,
       r.train_end_time, r.past_train_end_time,
       r.list_size, r.metric, r.parameter,
       r.adj_value AS value,
       r.adj_max_recall_ratio AS max_recall_ratio,
       r.adj_recall_white_to_nonwhite AS recall_w_to_nw,
       r.adj_recall_nonwhite_to_white AS recall_nw_to_w
FROM {{schema}}.model_adjustment_results_race_2way r
JOIN base_mgs b
  ON r.model_group_id = b.model_group_id
  AND r.past_train_end_time = b.train_end_time
  AND r.list_size = b.list_size
  AND r.metric = b.metric
  AND r.parameter = b.parameter
WHERE r.train_end_time > r.past_train_end_time

UNION ALL

-- Model selection on last time after adjustment, with adjustment applied
SELECT 'Best Adjusted Metric - Adjusted'::VARCHAR(128) AS strategy,
       r.train_end_time, r.past_train_end_time,
       r.list_size, r.metric, r.parameter,
       r.adj_value AS value,
       r.adj_max_recall_ratio AS max_recall_ratio,
       r.adj_recall_white_to_nonwhite AS recall_w_to_nw,
       r.adj_recall_nonwhite_to_white AS recall_nw_to_w
FROM {{schema}}.model_adjustment_results_race_2way r
JOIN adj_mgs b
  ON r.model_group_id = b.model_group_id
  AND r.past_train_end_time = b.train_end_time
  AND r.list_size = b.list_size
  AND r.metric = b.metric
  AND r.parameter = b.parameter
WHERE r.train_end_time > r.past_train_end_time

UNION ALL

-- Composite model
SELECT 'Composite Model - Adjusted'::VARCHAR(128) AS strategy,
      r.train_end_time AS train_end_time,
      r.past_train_end_time AS past_train_end_time,
      r.list_size, metric, parameter,
      r.value AS value,
      r.max_recall_ratio AS max_recall_ratio,
      r.recall_white_to_nonwhite AS recall_w_to_nw,
      r.recall_nonwhite_to_white AS recall_nw_to_w
FROM {{schema}}.composite_results_race_2way r
WHERE train_end_time > past_train_end_time
;
""")

params = {}
params['schema'] = 'hemank_bias_fexp_orig'
sql_orig = query.render(**params)
ts_df_orig = pd.read_sql(sql_orig, conn)

params['schema'] = 'hemank_bias_fexp_us'
sql_us = query.render(**params)
ts_df_us = pd.read_sql(sql_us, conn)

params['schema'] = 'hemank_bias_fexp_us_frac'
sql_us_frac = query.render(**params)
ts_df_us_frac = pd.read_sql(sql_us_frac, conn)

params['schema'] = 'hemank_bias_fexp_os'
sql_os = query.render(**params)
ts_df_os = pd.read_sql(sql_os, conn)

params['schema'] = 'hemank_bias_fexp_os_frac'
sql_os_frac = query.render(**params)
ts_df_os_frac = pd.read_sql(sql_os_frac, conn)

params['schema'] = 'hemank_bias_fexp_us_2'
sql_us_2 = query.render(**params)
ts_df_us_2 = pd.read_sql(sql_us_2, conn)

params['schema'] = 'hemank_bias_fexp_us_frac_2'
sql_us_frac_2 = query.render(**params)
ts_df_us_frac_2 = pd.read_sql(sql_us_frac_2, conn)

params['schema'] = 'hemank_bias_fexp_us_3'
sql_us_3 = query.render(**params)
ts_df_us_3 = pd.read_sql(sql_us_3, conn)

params['schema'] = 'hemank_bias_fexp_us_frac_3'
sql_us_frac_3 = query.render(**params)
ts_df_us_frac_3 = pd.read_sql(sql_us_frac_3, conn)